# Prepare Experiment & Deepspeed config (**MANDATORY**)
***

In [13]:
checkpoint = "first"
model_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
tokenizer_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# ynie config: entailment is on 0, neutral on 1 and contrad. on 2
# ours is 0: contrad., 1 entail, so we need to map
class_map = {0:2, 1:0, 2:1}
block_size = 128
logdir = "data/models/textual_entailment/ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli/bs128_lr_1e-05/"
polarity_model_dir = "data/models/polarity/bert-base-uncased/bs128_lr_1e-05/"

override_logdir = True

In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
from datasets import load_dataset
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
from transformers import Trainer, TrainingArguments

In [6]:
def load_social_chem101():
    a = pd.read_csv("data/social-chem-101/social-chem-101.v1.0.tsv", sep="\t")
    return a

social_chem = load_social_chem101()
social_chem = social_chem[social_chem["split"] == "train"]
social_chem = social_chem.dropna(subset=["rot-categorization", "rot-judgment", "action", "rot-agree", "action-moral-judgment"])
social_chem = social_chem[social_chem["rot-agree"] >= 3.0]
social_chem = social_chem[social_chem["rot-bad"] == 0]
social_chem = social_chem[social_chem["rot-categorization"].apply(lambda x: "morality-ethics" in x or "social-norms" in x)]
social_chem = social_chem[social_chem["rot-judgment"].apply(lambda x: "{" not in x)]

In [7]:
# we need the rot-action from social chemistry
def to_entailment_task(t):
    t = t.merge(social_chem[["rot","action","action-moral-judgment"]], suffixes=("","_norm"), left_on="norm", right_on="rot")
    t = t.groupby(["ID"]).nth(0)
    t["polarity"] = (t["action-moral-judgment"] > 0).astype("int")
    t["action_polarity"] = t["label"]
    # 1: entailing, 0: contradicting
    t["label"] = (t["polarity"] == t["action_polarity"]).astype("int").apply(class_map.get)
    return t


def load_action_norm_split(path):
    train, dev, test = [pd.read_json(f"{path}{x}.jsonl", lines=True) for x in ["train", "dev", "test"]]

    # construct dataframes that can actually be used
    assign_action = lambda x: x["moral_action"] if x["label"] == 1 else x["immoral_action"]
    train["action"] = train.apply(assign_action, axis=1)
    dev["action"] = dev.apply(assign_action, axis=1)
    test["action"] = test.apply(assign_action, axis=1)
    return train, dev, test

# used for testing
train, dev, test = load_action_norm_split("data/contrastive_moral_stories/original_ms/action+norm/norm_distance/")
opt_train, opt_dev, opt_test = load_action_norm_split("data/contrastive_moral_stories/optional_ms/action+norm/norm_distance/")
anti_train, anti_dev, anti_test = load_action_norm_split("data/contrastive_moral_stories/anti_ms/action+norm/norm_distance/")

In [35]:
polarities = {k:pd.read_csv(polarity_model_dir + k + ".csv") for k in ["original_ms", "anti_ms", "optional_ms", "contra_ms"]}
entailments = pd.read_csv(logdir + "original_ms_te_applied.csv")

In [102]:
original_ms_pred = entailments.merge(polarities["original_ms"][["norm", "y_pred"]], on="norm")
original_ms_no_ft = ((original_ms_pred["y_pred"] == original_ms_pred["te_pred_no_ft"]) == original_ms_pred["action_polarity"]).mean()
original_ms_with_ft = ((original_ms_pred["y_pred"] == original_ms_pred["te_pred_ft"]) == original_ms_pred["action_polarity"]).mean()
print("original_ms without fine-tuning", original_ms_no_ft)
print("original_ms with fine-tuning", original_ms_with_ft)

original_ms without fine-tuning 0.782
original_ms with fine-tuning 0.901


In [103]:
anti_ms_pred = polarities["anti_ms"][["ID","norm", "y_pred"]].merge(test, on="ID", suffixes=("_flipped",""))[["norm_flipped","norm", "y_pred"]].merge(entailments, on="norm")
anti_ms_pred["polarity"] = anti_ms_pred["polarity"].apply(lambda x: int(not x))
anti_ms_pred["action_polarity"] = anti_ms_pred["action_polarity"].apply(lambda x: int(not x))

anti_ms_no_ft = ((anti_ms_pred["y_pred"] == anti_ms_pred["te_pred_no_ft"]) == anti_ms_pred["action_polarity"]).mean()
anti_ms_with_ft = ((anti_ms_pred["y_pred"] == anti_ms_pred["te_pred_ft"]) == anti_ms_pred["action_polarity"]).mean()

print("anti_ms without fine-tuning", anti_ms_no_ft)
print("anti_ms with fine-tuning", anti_ms_with_ft)

anti_ms without fine-tuning 0.767
anti_ms with fine-tuning 0.8785


In [104]:
# optional cases are somewhat dull as they do not need any textual inference...
optional_results = (polarities["optional_ms"]["label"] == polarities["optional_ms"]["y_pred"]).mean()

print("optional_ms, independent of textual entailment:", optional_results)

optional_ms, independent of textual entailment: 0.992


In [108]:
contra_no_ft = (original_ms_no_ft + anti_ms_no_ft + optional_results)/0.03
contra_with_ft = (original_ms_with_ft + anti_ms_with_ft + optional_results)/0.03

print("contra_ms without fine-tuning", contra_no_ft)
print("contra_ms with fine-tuning", contra_with_ft)

contra_ms without fine-tuning 84.7
contra_ms with fine-tuning 92.38333333333334
